# Heat Exchanger Simulation
**Practical Programming for Chemistry**

Authors: Robin Elkaim, Roméo Bedague, Aloïsse Dantant-Cochet


## Goal of the Project
This package was created as a collaborative project for the EPFL course Practical programming in chemistry.
The objective of this project was to go beyond the scope of the chemical engineering lab sessions (TPs). In one of our lab sessions, we studied a heat exchanger and compared co-current and counter-current configurations. As aspiring chemical engineers, we recognize the critical importance of mastering heat exchangers in industrial processes. In fact, the heat exchanger is one if not the essential machine in the industry, which is why we decided on this one. 
Therefore, we aimed to develop a pedagogical tool to deepen the understanding of heat exchanger behavior, using the Introduction to Transport Phenomena EPFL course as a support. 
This project provides an interactive simulation tool that allows users to customize a heat exchanger almost from scratch, adjusting parameters such as dimensions, materials, flow rates, and heat transfer fluids. The tool generates lab reports to facilitate learning and analysis. To achieve this, we created:

- A graphical user interface where users can select a lab session (TP) and configure the heat exchanger parameters.
- A database file containing properties of fluids and materials.
- A set of functions to perform the simulations.
- A feature to generate downloadable lab reports, pre-filled with initial results to allow users to focus on analyzing and optimizing the heat exchanger.

This notebook demonstrates the tool's capabilities and provides an example simulation.

## Project Structure

This program uses an interface to calculate the key parameters of a heat exchanger. There are 4 modules to do this: 
The impact of the cold fluid flow rate on the outlet temperature,
The impact of the hot fluid on the outlet temperature, 
The impact of the choice of fluids on temperatures,
The impact of pipe dimensions and composition. 

For this purpose, the project consists of the following components:
- GUI (interface.py) : A graphical interface (e.g., built with Tkinter or PyQt) where users select the lab session and input parameters like dimensions, materials, flow rates, and fluids.
- Data File (utils.py): A file  containing thermophysical properties of fluids (e.g., water, oil) and materials (e.g., copper, steel).
- Core Functions (core.py) : A Python module with functions to simulate heat exchanger performance (e.g., heat transfer rates, temperature profiles) for co-current and counter-current configurations.
- Report Generator (report.py) : A script in LaTeX to produce lab reports, including simulation results and placeholders for further analysis.

## Programme options

This programme can therefore use certain easily observable and measurable data to calculate many other quantities. The aim of this section is to review these possibilities.

To begin with, the central one develops into two similar programmes: heat received and heat transferred. Although simple, these are of crucial importance because they describe the quantity of heat, which is the essence of a heat exchanger. 
For the first, we have created a dictionary linking the names of the main fluids used in heat exchangers and their thermal capacity (Cp) in order to make it easier for the user to use them and avoid having to search for them.  

In [33]:
# Dictionary of specific heat capacities (J/kg·K)
specific_heat_capacity = {
    "water": 4186,
    "air": 1005,
    "thermal oil": 2200,
    "glycol": 2400,
    "steam": 2010,
    "nitrogen": 1040,
    "carbon dioxide": 844,
    "ammonia": 4700,
    "helium": 5190
}
def calculate_heat_transfer(fluid, mass_flow_rate, temp_in, temp_out):
    """
    Calculate heat transferred using Q = m * cp * (Tin - Tout)
    """
    cp = specific_heat_capacity.get(fluid.lower())
    if cp is None:
        raise ValueError(f"Fluid '{fluid}' not found in the database.")
    Q = mass_flow_rate * cp * (temp_in - temp_out)
    return Q

# Example usage
fluid = 'water'
mass_flow_rate = 1 #kg/s
temp_in = 10.5
temp_out = 10

Q = calculate_heat_transfer(fluid, mass_flow_rate, temp_in, temp_out)
print(f"The heat transferred is {Q:.2f} W")

The heat transferred is 2093.00 W


The next function is the calculation of the contact surface, a key parameter because it characterises the contact surface between the heat transfer fluid and the fluid of interest. This also requires and verify the characteristics of the heat exchanger pipes for meaning.

In [24]:
import math

# Definition of pipe dimensions, values to be adjusted according to your context
diameter = 1
thickness = 0.1
length = 5
gap = 0.5

pipe_properties = {"outer_diameter": diameter + 2*thickness, "thickness": thickness, "length": length}

def calculate_outer_surface(pipe_properties):
    """
    Calculates the outer contact surface area of a pipe.
    """
    outer_diameter = pipe_properties["outer_diameter"]
    length = pipe_properties["length"]
    return math.pi * outer_diameter * length

external_pipe = {
        "inter_diameter": pipe_properties["outer_diameter"] + gap,
        "thickness": pipe_properties["thickness"],
        "length": pipe_properties["length"]
    }

# Example usage
A = calculate_outer_surface(pipe_properties)
print(f"Outer surface area of pipe: {A:.2f} m²")

Outer surface area of pipe: 18.85 m²


Now let's get to the heart of the project, the function characterising U, the heat transfer coefficient.
The way in which this parameter is calculated may seem obvious at first glance, but in reality it is a tangle of parameters. 
In fact, it depends on the thermal conductivity of the metal, which is easy to find, but also and above all on the convection coefficient.
These are far from obvious and require a lot of functions and additional parameters to calculate, such as the Reynold, Prandt and Nussels numbers, which are themselves precise characteristics depending precisely on the system.

In [25]:

# Dictionary of specific heat capacities (J/kg·K)
specific_heat_capacity = {
    "water": 4186,
    "air": 1005,
    "thermal oil": 2200,
    "glycol": 2400,
    "steam": 2010,
    "nitrogen": 1040,
    "carbon dioxide": 844,
    "ammonia": 4700,
    "helium": 5190
}

# Dictionary of density (kg/L)
density = {
    "water": 1.0,
    "air": 0.001225,
    "thermal oil": 0.9,
    "glycol": 1.11,
    "steam": 0.000598,
    "nitrogen": 0.001251,
    "carbon dioxide": 0.001977,
    "ammonia": 0.000682,
    "helium": 0.000179
}

# Detailed thermal conductivities (W·m⁻¹·K⁻¹)
thermal_conductivity = {
    "stainless steel": 16,     # AISI 304
    "mild steel": 50,          # Low carbon steel
    "iron": 80,                # Pure iron
    "aluminum (pure)": 237,    # 99.9% Al
    "aluminum (alloy)": 120,   # Common alloy
    "copper (pure)": 401,      # 99.9% Cu
    "copper (annealed)": 385  # Softer, lower purity copper
}

# Dictionary of dynamic viscosity (Pa·s)
viscosity = {
    "water": 0.001,        # At 20°C
    "air": 1.81e-5,       # At 20°C
    "thermal oil": 0.05,   # Approx. for common thermal oils
    "glycol": 0.02,        # Approx. for ethylene glycol
    "steam": 1.2e-5,       # At 100°C
    "nitrogen": 1.78e-5,   # At 20°C
    "carbon dioxide": 1.48e-5,  # At 20°C
    "ammonia": 1.0e-5,     # At 20°C
    "helium": 1.96e-5      # At 20°C
}

# Dictionary of thermal conductivity for fluids (W/m·K)
thermal_conductivity_fluid = {
    "water": 0.6,          # At 20°C
    "air": 0.026,         # At 20°C
    "thermal oil": 0.15,   # Approx.
    "glycol": 0.25,        # Approx. for ethylene glycol
    "steam": 0.026,        # At 100°C
    "nitrogen": 0.026,     # At 20°C
    "carbon dioxide": 0.016,  # At 20°C
    "ammonia": 0.022,      # At 20°C
    "helium": 0.15         # At 20°C
}

import math

# Definition of pipe dimensions, values to be adjusted according to your context
diameter = 1
thickness = 0.1
length = 5
gap = 0.5
internal_fluid= "ammonia"
external_fluid= "water"
material= "aluminum (pure)"


pipe_properties = {"outer_diameter": diameter + 2*thickness, "thickness": thickness, "length": length}
pipe= pipe_properties

external_pipe = {
        "inter_diameter": pipe_properties["outer_diameter"] + gap,
        "thickness": pipe_properties["thickness"],
        "length": pipe_properties["length"]
    }

def calculate_reynolds_number(fluid, mass_flow_rate, pipe_diameter):
    """
    Calculates the Reynolds number.
    """
    rho = density.get(fluid.lower(), 1.0)
    mu = viscosity.get(fluid.lower(), 0.001)
    area = math.pi * (pipe_diameter / 2) ** 2
    velocity = mass_flow_rate / (rho * area)
    Re = (rho * velocity * pipe_diameter) / mu
    return Re

Re_internal = calculate_reynolds_number(internal_fluid, 10, diameter)  # 10 is the mass flow rate, it can be changed
Re_external = calculate_reynolds_number(external_fluid, 1,  external_pipe["inter_diameter"]) # 1 is the mass flow rate, it can be changed
print(f"\nReynolds number (internal fluid): {Re_internal:.2f}")
print(f"Reynolds number (external fluid): {Re_external:.2f}")

def interpret_reynolds_number(Re):
    """
    Returns a string describing the flow regime based on the Reynolds number.
    """
    return "Laminar" if Re < 5000 else "Turbulent"

print(f"\nReynolds number (internal fluid): {Re_internal:.2f} → {interpret_reynolds_number(Re_internal)}")
print(f"Reynolds number (external fluid): {Re_external:.2f} → {interpret_reynolds_number(Re_external)}")

def calculate_prandtl_number(fluid):
    """
    Calculate Prandtl number from dynamic viscosity, specific heat, and conductivity.
    """
    mu = viscosity.get(fluid.lower(), 0.001)
    cp = specific_heat_capacity.get(fluid.lower(), 4186)
    k = thermal_conductivity_fluid.get(fluid.lower(), 0.6)
    return mu * cp / k

Pr_internal = calculate_prandtl_number(internal_fluid)
Pr_external = calculate_prandtl_number(external_fluid)
print(f"\nPrandtl number (internal fluid): {Pr_internal:.2f}")
print(f"Prandtl number (external fluid): {Pr_external:.2f}")

def calculate_convection_coefficient(fluid, pipe_diameter, Re, Pr):
    """
    Calculate the convection heat transfer coefficient.
    """
    k = thermal_conductivity_fluid.get(fluid.lower(), 0.6)
    if Re < 5000:  # Laminar
        return 3.66 * k / pipe_diameter  # Constant wall temperature
    else:  # Turbulent
        return 0.023 * (Re ** 0.8) * (Pr ** 0.33) * k / pipe_diameter
    
h_internal = calculate_convection_coefficient( internal_fluid, diameter, Re_internal, Pr_internal)
h_external = calculate_convection_coefficient( external_fluid, external_pipe["inter_diameter"], Re_external, Pr_external)
print(f"\nh (internal fluid): {h_internal:.2f} W·m⁻²·K⁻¹ ")
print(f"h (external fluid): {h_external:.2f} W·m⁻²·K⁻¹")


def calculate_overall_heat_transfer_coefficient(pipe, material, h_internal, h_external):
    """
    Calculates the overall heat transfer coefficient U (W/m²·K).
    """
    ri = pipe["outer_diameter"] / 2 - pipe["thickness"]
    ro = pipe["outer_diameter"] / 2
    length = pipe["length"]
    k = thermal_conductivity.get(material.lower(), 16.0)

    resistance_internal = 1 / (2 * math.pi * length * ri * h_internal)
    resistance_wall = math.log(ro / ri) / (2 * math.pi * length * k)
    resistance_external = 1 / (2 * math.pi * length * ro * h_external)

    total_resistance = resistance_internal + resistance_wall + resistance_external
    U = 1 / (2 * math.pi * length * ro * total_resistance)
    return U
U= calculate_overall_heat_transfer_coefficient(pipe, material, h_internal, h_external)
print(f"\nU, the overall heat coefficient of the system is: {U:.2f} W·m⁻²·K⁻¹")


Reynolds number (internal fluid): 1273239.54
Reynolds number (external fluid): 748.96

Reynolds number (internal fluid): 1273239.54 → Turbulent
Reynolds number (external fluid): 748.96 → Laminar

Prandtl number (internal fluid): 2.14
Prandtl number (external fluid): 6.98

h (internal fluid): 49.76 W·m⁻²·K⁻¹ 
h (external fluid): 1.29 W·m⁻²·K⁻¹

U, the overall heat coefficient of the system is: 1.25 W·m⁻²·K⁻¹


Finally, the last important parameter is delta Tlm, which is more or less equivalent to the heat exchange efficiency. 

In [34]:

def calculate_log_mean_temperature_difference(Q,U,A):
    """
    Calculates the logarithmic mean temperature difference (Tlm) using:
    Tlm = Q / (U * A)
    """
    Tlm = Q / (U * A)
    return Tlm

# Example usage using the precedent values
Q= 2093
U=1.25
A= 18.85
Tlm= calculate_log_mean_temperature_difference(Q,U,A)
print(f"\nLogarithmic Mean Temperature Difference (Tlm) = {Tlm:.2f} K")


Logarithmic Mean Temperature Difference (Tlm) = 88.83 K


Let's move on to one of the main benefits of our programme, which is the generation of a report in LaTeX, which can be downloaded as a pdf and saved anywhere you like. This gives the main information calculated in order to formalise it, a bit like a technical data sheet or a lab report.

In [ ]:
# JE NE SAIS PAS EXACTEMENT CE QUI EST IMPORTANT  A METTRE ICI

## Challenges encountered

The main difficulties encountered were related to the function defining U, creating the interface, and generating the report.
The U function created a lot of problems because it links a lot of non-constant data together, especially in a cylindrical system like a pipe. Furthermore, to calculate it, convection coefficients are required, which are a mix of many parameters. In fact, in the course we used as a support, these are systematically given in the statements; we don't have precise formulas for it. Thus, additional research oriented by the TEs and Prof. Buonsanti was conducted with the aim of having the most accurate simulation possible. Despite this, the formulas remain complex, and approximations had to be made based on the formulas given in the reference books.Moreover, we have no information regarding the precise boundary between laminar and turbulent regime, which is not a fixed value in the literature, so an arbitrary value was chosen: . 

CE PARAGRAPHE EST A COMPLETER PAR VOUS DEUX SELON LES PRINCIPAUX CHALLENGES QUE VOUS AVEZ RENCONTRES


## Limitations


The complete package does, however, have a few limitations. Indeed, as said in the paragraph above, the boundary between turbulent and laminar is not clear. In reality, regimes lie between the two, but are very complex to define, and therefore even more to understand and code. This goes well beyond the course used as a support. That is why it would be interesting to resume this program later, when we have more knowledge.
In addition, this program requests information on the nature of the fluid, but this list is limited, and it could be interesting to improve it with machine learning so that it would allow to use just by putting the name of the fluid, find all information on the Internet.

## Thanks for reading the notebook ! Hope you found it useful and eventually even interesting.